## Overview

This notebook is designed to provide a broad overview of Hail's functionality, with emphasis on the functionality to manipulate and query a genetic dataset. We walk through a genome-wide SNP association test, and demonstrate the need to control for confounding caused by population stratification.


In [ ]:
import hail as hl
import hail.expr.aggregators as agg
hl.init()

If the above cell ran without error, we're ready to go! 

Before using Hail, we import some standard Python libraries for use throughout the notebook.

In [ ]:
from pprint import pprint
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span
output_notebook()

## Check for tutorial data or download if necessary

This cell downloads the necessary data if it isn't already present.

In [ ]:
hl.utils.get_1kg('data/')

## Loading data from disk

Hail has its own internal data representation, called a MatrixTable. This is both an on-disk file format and a [Python object](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable). Here, we read a MatrixTable from disk.

This dataset was created by downsampling a public 1000 genomes VCF to about 50 MB.

In [ ]:
ds = hl.read_matrix_table('data/1kg.mt')

## Getting to know our data

It's important to have easy ways to slice, dice, query, and summarize a dataset. Some of these methods are demonstrated below.

The [rows](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.rows) method can be used to get a table with all the row fields in our MatrixTable. 

We can use `rows` along with [select](https://hail.is/docs/devel/hail.Table.html#hail.Table.select) to pull out 5 variants. The `select` method takes either a string refering to a field name in the table, or a Hail [Expression](https://hail.is/docs/devel/expr/expression.html#hail.expr.expression.Expression). Here, we leave the arguments blank to keep only the row key fields, `locus` and `alleles`.

Use the `show` method to display the variants.

In [ ]:
ds.rows().select().show(5)

Here is how to peek at the first few sample IDs:

In [ ]:
ds.s.show(5)

To look at the first few genotype calls, we can use [entries](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.entries) along with `select` and `take`. The `take` method collects the first n rows into a list. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format. 

Try changing `take` to `show` in the cell below.

In [ ]:
ds.entry.take(5)

## Adding column fields

A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. Column fields are where you'll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis. 

In this tutorial, we demonstrate how to take a text file and use it to annotate the columns in a MatrixTable. 

The file provided contains the sample ID, the population and "super-population" designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

This file can be imported into Hail with [import_table](https://hail.is/docs/devel/methods/impex.html#hail.methods.import_table). This method produces a [Table](https://hail.is/docs/devel/hail.Table.html#hail.Table) object. Think of this as a Pandas or R dataframe that isn't limited by the memory on your machine -- behind the scenes, it's distributed with Spark.

In [ ]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))

A good way to peek at the structure of a `Table` is to look at its `schema`. 



In [ ]:
table.describe()

To peek at the first few values, use the `show` method:

In [ ]:
table.show(width=100)

Now we'll use this table to add sample annotations to our dataset, storing the annotations in column fields in our MatrixTable. First, we'll print the existing column schema:

In [ ]:
print(ds.col.dtype)

We use the [annotate_cols](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) method to join the table with the MatrixTable containing our dataset.

In [ ]:
ds = ds.annotate_cols(**table[ds.s])

In [ ]:
print(ds.col.dtype)

In [ ]:
print(ds.col.dtype.pretty())

## Query functions and the Hail Expression Language

Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments.

We will start by looking at some statistics of the information in our table. The [aggregate](https://hail.is/docs/devel/hail.Table.html#hail.Table.aggregate) method can be used to aggregate over rows of the table.

`counter` is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.

In [ ]:
pprint(table.aggregate(agg.counter(table.SuperPopulation)))

`stats` is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [ ]:
pprint(table.aggregate(agg.stats(table.CaffeineConsumption)))

However, these metrics aren't perfectly representative of the samples in our dataset. Here's why:

In [ ]:
table.count()

In [ ]:
ds.count_cols()

Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use [aggregate_cols](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) to get the metrics for only the samples in our dataset.

In [ ]:
ds.aggregate_cols(agg.counter(ds.SuperPopulation))

In [ ]:
pprint(ds.aggregate_cols(agg.stats(ds.CaffeineConsumption)))

The functionality demonstrated in the last few cells isn't anything especially new: it's certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like `awk`. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants. 

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base). 

To do this, we need to get the alternate allele of each variant and then count the occurences of each unique ref/alt pair. This can be done with Hail's `counter` method.

In [ ]:
snp_counts = ds.aggregate_rows(agg.counter(hl.Struct(ref=ds.alleles[0], alt=ds.alleles[1])))
pprint(snp_counts)

We can list the counts in descending order using Python's Counter class.

In [ ]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

It's nice to see that we can actually uncover something biological from this small dataset: we see that these frequencies come in pairs. C/T and G/A are actually the same mutation, just viewed from from opposite strands. Likewise, T/A and A/T are the same mutation on opposite strands. There's a 30x difference between the frequency of C/T and A/T SNPs. Why?

The same Python, R, and Unix tools could do this work as well, but we're starting to hit a wall - the latest [gnomAD release](http://gnomad.broadinstitute.org/) publishes about 250 million variants, and that won't fit in memory on a single computer.

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 284 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about **5 trillion** unique genotypes.

Hail plotting functions allow Hail fields as arguments, so we can pass in the DP field directly here. If the range and bins arguments are not set, this function will compute the range based on minimum and maximum values of the field and use the default 50 bins.

In [ ]:
p = hl.plot.histogram(ds.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
show(p)

## Quality Control

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no "push-button" solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [sample_qc](https://hail.is/docs/devel/methods/genetics.html#hail.methods.sample_qc) method, which produces a set of useful metrics and stores them in a column field.

In [ ]:
ds.col.describe()

In [ ]:
ds = hl.sample_qc(ds)

In [ ]:
ds.col.describe()

Plotting the QC metrics is a good place to start.

In [ ]:
p = hl.plot.histogram(ds.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

In [ ]:
p = hl.plot.histogram(ds.sample_qc.gq_mean, range=(10,70), legend='Mean Sample GQ')
show(p)

Often, these metrics are correlated.

In [ ]:
p = hl.plot.scatter(ds.sample_qc.dp_mean, ds.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

Removing outliers from the dataset will generally improve association results. We can draw lines on the above plot to indicate outlier cuts.

In [ ]:
p.renderers.extend(
    [Span(location=0.97, dimension='width', line_color='black', line_width=1),
     Span(location=4, dimension='height', line_color='black', line_width=1)])
show(p)

We'll want to remove all samples that fall in the bottom left quadrant. 

It's easy to filter when we've got the cutoff values decided:

In [ ]:
ds = ds.filter_cols((ds.sample_qc.dp_mean >= 4) & (ds.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % ds.count_cols())

Next is genotype QC. To start, we'll print the post-sample-QC call rate. It's actually gone _up_ since the initial summary - dropping low-quality samples disproportionally removed missing genotypes.

Import the `hail.expr.functions` module.

In [ ]:
call_rate = ds.aggregate_entries(agg.fraction(hl.is_defined(ds.GT)))
print('pre QC call rate is %.3f' % call_rate)

It's a good idea to filter out genotypes where the reads aren't where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In [ ]:
ab = ds.AD[1] / hl.sum(ds.AD)

filter_condition_ab = ((ds.GT.is_hom_ref() & (ab <= 0.1)) |
                        (ds.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (ds.GT.is_hom_var() & (ab >= 0.9)))

ds = ds.filter_entries(filter_condition_ab)

In [ ]:
post_qc_call_rate = ds.aggregate_entries(agg.fraction(hl.is_defined(ds.GT)))
print('post QC call rate is %.3f' % post_qc_call_rate)

Variant QC is a bit more of the same: we can use the [variant_qc](https://hail.is/docs/devel/methods/genetics.html?highlight=variant%20qc#hail.methods.variant_qc) method to produce a variety of useful statistics, plot them, and filter.

In [ ]:
ds.row.describe()

The [cache](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.cache) is used to optimize some of the downstream operations.

In [ ]:
ds = hl.variant_qc(ds).cache()

In [ ]:
ds.row.describe()

In [ ]:
qc_table = ds.rows()
qc_table = qc_table.select(gq_mean = qc_table.variant_qc.gq_stats.mean,
                           call_rate = qc_table.variant_qc.call_rate,
                           p_hwe = qc_table.variant_qc.p_hwe,
                           AAF = qc_table.variant_qc.AF[1],)

p1 = hl.plot.histogram(qc_table.gq_mean, range=(10,80), legend='Variant Mean GQ')

p2 = hl.plot.histogram(qc_table.AAF, range=(0,1), legend='Alternate Allele Frequency')

p3 = hl.plot.histogram(qc_table.call_rate, range=(.5,1), legend='Variant Call Rate')

p4 = hl.plot.histogram(qc_table.p_hwe, range=(0,1), legend='Hardy-Weinberg Equilibrium p-value')

g = gridplot([[p1, p2], [p3, p4]])
show(g)

These statistics actually look pretty good: we don't need to filter this dataset. Most datasets require thoughtful quality control, though. The [filter_rows](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.filter_rows) method can help!

## Let's do a GWAS!

First, we need to restrict to variants that are : 

 - common (we'll use a cutoff of 1%)
 - uncorrelated (not in linkage disequilibrium)


In [ ]:
common_ds = ds.filter_rows(ds.variant_qc.AF[1] > 0.01)

In [ ]:
print('Samples: %d  Variants: %d' % (common_ds.count_cols(), common_ds.count_rows()))

These filters removed about 15% of sites (we started with a bit over 10,000). This is _NOT_ representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we'd expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we've already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [ ]:
gwas = hl.linear_regression(y=common_ds.CaffeineConsumption, x=common_ds.GT.n_alt_alleles())
gwas.row.describe()

Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

Hail makes it easy to make a [Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q–Q_plot).

In [ ]:
p = hl.plot.qq(gwas.linreg.p_value)
show(p)

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there's a confounder.

We didn't tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression. 

The [linear_regression](https://hail.is/docs/devel/methods/stats.html#hail.methods.linear_regression) method can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don't have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The [pca](https://hail.is/docs/devel/methods/stats.html#hail.methods.pca) method produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The [hwe_normalized_pca](https://hail.is/docs/devel/methods/genetics.html#hail.methods.hwe_normalized_pca) method does the same, using HWE-normalized genotypes for the PCA.

In [ ]:
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_ds.GT)

In [ ]:
pprint(pca_eigenvalues)

In [ ]:
pca_scores.show(5, width=100)

Now that we've got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [ ]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_ds.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

Now we can rerun our linear regression, controlling for sample sex and the first few principal components. We'll do this with input variable the number of alternate alleles as before, and again with input variable the genotype dosage derived from the PL field.

In [ ]:
cds = common_ds.annotate_cols(pca = pca_scores[common_ds.s])

linear_regression_results = hl.linear_regression(
    y=cds.CaffeineConsumption, x=cds.GT.n_alt_alleles(),
    covariates=[cds.isFemale, cds.pca.scores[0], cds.pca.scores[1], cds.pca.scores[2]])

In [ ]:
p = hl.plot.qq(linear_regression_results.linreg.p_value)
show(p)

We can also use the genotype dosage (the probability-weighted expected number of alternate alleles) instead of the hard call in our regression

In [ ]:
def plDosage(pl):
    linear_scaled = pl.map(lambda x: 10 ** (-x / 10))
    pl_sum = hl.sum(linear_scaled)
    normed = linear_scaled / pl_sum
    return 1 * normed[1] + 2 * normed[2]

In [ ]:
linear_regression_results = hl.linear_regression(
    y=cds.CaffeineConsumption, x=plDosage(cds.PL),
    covariates=[cds.isFemale, cds.pca.scores[0], cds.pca.scores[1], cds.pca.scores[2]])

In [ ]:
p = hl.plot.qq(linear_regression_results.linreg.p_value)
show(p)

That's more like it! We may not be publishing ten new coffee-drinking loci in _Nature_, but we shouldn't expect to find anything but the strongest signals from a dataset of 284 individuals anyway. 

## Rare variant analysis

Here we'll demonstrate how one can use the expression language to group and count by any arbitrary properties in row and column fields. Hail also implements the sequence kernel association test (SKAT).


In [ ]:
entries = ds.entries()
results = (entries.group_by(pop = entries.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = agg.count_where(entries.GT.is_het())))

In [ ]:
results.show()

What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?

In [ ]:
entries = entries.annotate(maf_bin = hl.cond(entries.info.AF[0]<0.01, "< 1%", 
                             hl.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.PurpleHair)
      .aggregate(mean_gq = agg.stats(entries.GQ).mean, 
                 mean_dp = agg.stats(entries.DP).mean))

In [ ]:
results2.show()

We've shown that it's easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

 - Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint
 - Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease

## Epilogue

Congrats! You've reached the end of the first tutorial. To learn more about Hail's API and functionality, take a look at the other tutorials. You can check out the [Python API](https://hail.is/docs/devel/api.html#python-api) for documentation on additional Hail functions. If you use Hail for your own science, we'd love to hear from you on [Zulip chat](https://hail.zulipchat.com) or the [discussion forum](http://discuss.hail.is).

For reference, here's the full workflow to all tutorial endpoints combined into one cell. It may take a minute! It's doing a lot of work.

In [ ]:
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

ds = hl.read_matrix_table('data/1kg.mt')
ds = ds.annotate_cols(**table[ds.s])
ds = hl.sample_qc(ds)
ds = ds.filter_cols((ds.sample_qc.dp_mean >= 4) & (ds.sample_qc.call_rate >= 0.97))
ab = ds.AD[1] / hl.sum(ds.AD)
filter_condition_ab = ((ds.GT.is_hom_ref() & (ab <= 0.1)) |
                        (ds.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (ds.GT.is_hom_var() & (ab >= 0.9)))
ds = ds.filter_entries(filter_condition_ab)
ds = hl.variant_qc(ds)
common_ds = ds.filter_rows(ds.variant_qc.AF[1] > 0.01)

pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_ds.GT)

cds = common_ds.annotate_cols(pca = pca_scores[common_ds.s])
linear_regression_results = hl.linear_regression(
    y=cds.CaffeineConsumption, x=cds.GT.n_alt_alleles(),
    covariates=[cds.isFemale, cds.pca.scores[0], cds.pca.scores[1], cds.pca.scores[2]])
pvals = linear_regression_results.linreg.p_value.collect()